In [2]:
import requests
from bs4 import BeautifulSoup

def parse_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Заголовок статьи: сначала ищем <h1>, если нет - <title>, <h2>
    title_tag = soup.find('h1') or soup.find('title') or soup.find('h2')
    title = title_tag.get_text(strip=True) if title_tag else ''

    # Основной текст статьи
    paragraphs = []
    # Стандартный вариант — внутри <article>
    article_zone = soup.find('article')
    if article_zone:
        paragraphs = article_zone.find_all('p')
    else:
        # fallback: берем div с макс. количеством <p>
        divs = soup.find_all('div')
        best_div = max(divs, key=lambda div: len(div.find_all('p')), default=None)
        if best_div and len(best_div.find_all('p')) > 3:
            paragraphs = best_div.find_all('p')
        # fallback на все <p>
        if not paragraphs:
            paragraphs = soup.find_all('p')

    text = ' '.join([p.get_text(strip=True) for p in paragraphs])

    # Возврат универсальной модели
    return {"title": title, "text": text}

# Пример использования
url = "https://www.al.com/news/2025/10/major-pharmacy-chain-closes-all-locations-nationwide.html"
print(parse_article(url))

{'title': 'Major pharmacy chain closes all locations nationwide', 'text': 'Once a staple alongside CVS and Walgreens, a national pharmacy chain is no more. Rite Aid has officially closed all its locations nationwide,according to an announcement on its website. “All Rite Aid stores have now closed. We thank our loyal customers for their many years of support,” the announcement said. The website also has information for former customers on requesting prescription and immunization records and a link that allows people to search for a new pharmacy. The closures mark a dramatic turn in the final chapter of the once iconic chain. In 2022, Rite Aid operated 2,451 locations. By September 2, 2025,only 89 stores were still openand those have since closed. The final stores – located in Washington and Oregon – closed Sept. 29. Founded in Pennsylvania in 1962, Rite Aid was once the third-largest drug store chain in the U.S. Itfirst filed for bankruptcy in October 2023 due to high debt, declining re

In [4]:
def build_prompt(title, text):
    prompt = (f'''
        Analyze the following financial news article and return the result in the following Python dictionary structure:
        headline — a brief headline;
        hotness ∈ [0,1] — the hotness score;
        why_now — 1–2 sentences explaining why this is important now (novelty, confirmations, affected assets);
        entities — companies/tickers/countries/sectors mentioned in the article;
        timeline — key time markers (first mention → confirmation → clarification);
        draft — a draft for a post/cover note (headline, lead paragraph, 3 bullets, quote/note);
        dedup_group — ID of duplicate-cluster/reprints.
        ARTICLE TITLE: {title}
        ARTICLE TEXT: {text}
        Return a Python dictionary where keys are headline, hotness, why_now, entities, timeline, draft, dedup_group. Respond in English only.
        '''
    )
    return prompt

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

url = "https://www.al.com/news/2025/10/major-pharmacy-chain-closes-all-locations-nationwide.html"
article = parse_article(url)
prompt = build_prompt(article['title'], article['text'])

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>",
    "X-Title": "<YOUR_SITE_NAME>",
  },
  extra_body={},
  model="qwen/qwen3-vl-235b-a22b-thinking",
  messages = [
      { "role": "user", "content": prompt }
  ]
)

print(completion.choices[0].message.content)


JSONDecodeError: Expecting value: line 449 column 1 (char 2464)